# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información

El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.

Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Proporcionar un Conjunto de Datos:
    * Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.
    * Consultas: Define un conjunto de consultas específicas.
    * Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.

2. Calcular Resultados de Búsqueda:
    * Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.

3. Calcular las Métricas de Evaluación:
    * Calcular las siguientes métricas para cada sistema y consulta:
        * Precisión en el top-k (Prec@k)
        * Recall
        * F1-score
        * Mean Average Precision (MAP)
        * nDCG

4. Análisis y Comparación:
    * Comparar los resultados de los dos sistemas utilizando las métricas calculadas.
    * Discutir cuál sistema es más efectivo y por qué.

### Procedimiento
#### 1. Proporcionar un Conjunto de Datos

In [22]:
import re
import xml.etree.ElementTree as ET
import math

In [23]:
# Función para procesar el texto y extraer palabras clave, convirtiéndolas a minúsculas
# y eliminando caracteres no alfanuméricos
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Reemplazar caracteres no alfanuméricos por espacios
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

In [24]:
# Esta función carga el archivo XML, extrae los datos de cada documento
# y almacena el ID, título, palabras clave, autor y fecha en un diccionario
def parse_corpus(xml_file):
    # Parsear el archivo XML
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Diccionario para almacenar el corpus de documentos
    corpus = {}
    
    # Recorrer cada documento en el corpus XML
    for doc in root.findall('document'):
        # Extraer ID, título, palabras clave, autor y fecha de cada documento
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        
        # Procesar las palabras clave usando la función definida previamente
        keyword_set = process_text(keywords)
        
        # Almacenar los datos del documento en el diccionario de corpus
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [25]:
corpus = parse_corpus('../data/03ranking_corpus.xml')
for doc_id, doc_data in corpus.items():
    print(f"Documento ID: {doc_id}")
    for key, value in doc_data.items():
        print(f"  {key.capitalize()}: {value}")
    print("-" * 40)

Documento ID: 1
  Title: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
  Keywords: {'médica', 'tratamiento', 'salud', 'telemedicina', 'tecnología', 'crónica'}
  Author: Dr. Juan Pérez
  Date: 2023-01-15
----------------------------------------
Documento ID: 2
  Title: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
  Keywords: {'estudiantes', 'salud', 'mental', 'rendimiento', 'académico', 'nutrición'}
  Author: Dra. María López
  Date: 2023-02-10
----------------------------------------
Documento ID: 3
  Title: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
  Keywords: {'estudiantil', 'campus', 'amistad', 'relaciones', 'bienestar', 'sociales'}
  Author: Miguel Rodríguez
  Date: 2023-03-05
----------------------------------------
Documento ID: 4
  Title: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
  Keyword

In [26]:
# Paso 1.2: Definir un conjunto de consultas específicas
# En esta sección se define una lista de consultas que serán utilizadas
# para evaluar el sistema de recuperación de información
consultas = [
    "telemedicina salud crónica",
    "rendimiento académico salud mental",
    "tecnología y medicina preventiva",
    "bienestar emocional estudiantes universitarios",
    "impacto de la inteligencia artificial en comunidades remotas"
]

In [27]:
# Mostrar las consultas definidas
print("Consultas definidas:")
for i, consulta in enumerate(consultas, 1):
    print(f"Consulta {i}: {consulta}")


Consultas definidas:
Consulta 1: telemedicina salud crónica
Consulta 2: rendimiento académico salud mental
Consulta 3: tecnología y medicina preventiva
Consulta 4: bienestar emocional estudiantes universitarios
Consulta 5: impacto de la inteligencia artificial en comunidades remotas


In [28]:
# Paso 1.3: Juicios de Relevancia
# Aquí se define un conjunto de juicios de relevancia, donde se especifica qué documentos
# son relevantes para cada consulta. Esto sirve como base para evaluar la precisión y el recall
juicios_relevancia = {
    1: [1, 8],          # Documentos relevantes para la primera consulta
    2: [2, 7, 14],      # Documentos relevantes para la segunda consulta
    3: [10, 5, 8],      # Documentos relevantes para la tercera consulta
    4: [6, 8, 15],      # Documentos relevantes para la cuarta consulta
    5: [30, 1, 21]      # Documentos relevantes para la quinta consulta
}

In [29]:
# Mostrar los juicios de relevancia para cada consulta
print("\nJuicios de Relevancia:")
for consulta_id, documentos_relevantes in juicios_relevancia.items():
    print(f"Consulta {consulta_id}: Documentos relevantes: {documentos_relevantes}")


Juicios de Relevancia:
Consulta 1: Documentos relevantes: [1, 8]
Consulta 2: Documentos relevantes: [2, 7, 14]
Consulta 3: Documentos relevantes: [10, 5, 8]
Consulta 4: Documentos relevantes: [6, 8, 15]
Consulta 5: Documentos relevantes: [30, 1, 21]


#### 2. Calcular Resultados de Búsqueda

In [30]:
# Función para calcular la similitud de coseno entre dos conjuntos de palabras (sin TF-IDF)
def cosine_similarity(set1, set2):
    # Crear el conjunto de palabras únicas a partir de la unión de ambos conjuntos
    words = list(set1.union(set2))
    
    # Crear los vectores de frecuencia binarios para cada conjunto
    vec1 = [1 if word in set1 else 0 for word in words]
    vec2 = [1 if word in set2 else 0 for word in words]

    # Calcular el producto punto y las magnitudes de los vectores
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(v ** 2 for v in vec1))
    magnitude2 = math.sqrt(sum(v ** 2 for v in vec2))
    
    # Evitar la división por cero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    return dot_product / (magnitude1 * magnitude2)

# Función para calcular la similitud de Jaccard entre dos conjuntos de palabras
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:
        return 0.0
    return len(intersection) / len(union)

In [31]:
# Definir un diccionario para almacenar los resultados de ambos sistemas
resultados_sistemas = {'Sistema_Titulos': {}, 'Sistema_Keywords': {}}

In [32]:
for i, consulta in enumerate(consultas, 1):
    print(f"\nConsulta {i}: '{consulta}'")
    
    # Convertir la consulta en un conjunto de palabras
    consulta_set = process_text(consulta)
    
    # Resultados para Sistema 1 (basado en títulos)
    resultados_titulos = []
    for doc_id, doc_data in corpus.items():
        title_set = process_text(doc_data['title'])
        score = cosine_similarity(consulta_set, title_set)  # Usa cosine_similarity
        resultados_titulos.append((doc_id, score))
    
    # Ordenar y almacenar los resultados del Sistema 1
    resultados_ordenados_titulos = sorted(resultados_titulos, key=lambda x: x[1], reverse=True)
    resultados_sistemas['Sistema_Titulos'][i] = resultados_ordenados_titulos
    
    print("Resultados del Sistema basado en Títulos:")
    for doc_id, score in resultados_ordenados_titulos[:5]:  # Mostrar los 5 mejores resultados
        print(f"  Documento {doc_id} - Similitud: {score:.4f}")
    
    # Resultados para Sistema 2 (basado en palabras clave)
    resultados_keywords = []
    for doc_id, doc_data in corpus.items():
        keywords_set = doc_data['keywords']  # Las keywords ya son un conjunto en el corpus
        score = cosine_similarity(consulta_set, keywords_set)  # Usa cosine_similarity
        resultados_keywords.append((doc_id, score))
    
    # Ordenar y almacenar los resultados del Sistema 2
    resultados_ordenados_keywords = sorted(resultados_keywords, key=lambda x: x[1], reverse=True)
    resultados_sistemas['Sistema_Keywords'][i] = resultados_ordenados_keywords
    
    print("Resultados del Sistema basado en Palabras Clave:")
    for doc_id, score in resultados_ordenados_keywords[:5]:  # Mostrar los 5 mejores resultados
        print(f"  Documento {doc_id} - Similitud: {score:.4f}")


Consulta 1: 'telemedicina salud crónica'
Resultados del Sistema basado en Títulos:
  Documento 1 - Similitud: 0.3651
  Documento 2 - Similitud: 0.1601
  Documento 7 - Similitud: 0.1601
  Documento 14 - Similitud: 0.1491
  Documento 23 - Similitud: 0.1325
Resultados del Sistema basado en Palabras Clave:
  Documento 1 - Similitud: 0.7071
  Documento 2 - Similitud: 0.2357
  Documento 7 - Similitud: 0.2357
  Documento 11 - Similitud: 0.2357
  Documento 14 - Similitud: 0.2182

Consulta 2: 'rendimiento académico salud mental'
Resultados del Sistema basado en Títulos:
  Documento 2 - Similitud: 0.5547
  Documento 7 - Similitud: 0.5547
  Documento 14 - Similitud: 0.3873
  Documento 23 - Similitud: 0.2294
  Documento 13 - Similitud: 0.2236
Resultados del Sistema basado en Palabras Clave:
  Documento 2 - Similitud: 0.8165
  Documento 7 - Similitud: 0.8165
  Documento 14 - Similitud: 0.5669
  Documento 11 - Similitud: 0.4082
  Documento 18 - Similitud: 0.3780

Consulta 3: 'tecnología y medicina 

#### 3. Calcular las Métricas de Evaluación

In [33]:
# 3.1.1 Precisión en el top-k (Prec@k)
# Función para calcular Prec@k
def precision_at_k(resultados, relevantes, k):
    top_k = [doc_id for doc_id, _ in resultados[:k]]
    relevantes_encontrados = sum(1 for doc_id in top_k if doc_id in relevantes)
    return relevantes_encontrados / k

In [34]:
# 3.1.2 Recall
def recall(resultados, relevantes):
    relevantes_encontrados = sum(1 for doc_id, _ in resultados if doc_id in relevantes)
    return relevantes_encontrados / len(relevantes) if len(relevantes) > 0 else 0

In [35]:
# 3.1.3 F1-score
def f1_score(precision, recall):
    return (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

In [36]:
# 3.1.4 Mean Average Precision (MAP)
def mean_average_precision(resultados, relevantes):
    hits = 0
    sum_precisions = 0
    for i, (doc_id, _) in enumerate(resultados, 1):
        if doc_id in relevantes:
            hits += 1
            sum_precisions += hits / i
    return sum_precisions / len(relevantes) if len(relevantes) > 0 else 0

In [37]:
# 3.1.5 nDCG
# Función para calcular nDCG (Discounted Cumulative Gain normalizado)
def ndcg(resultados, relevantes):
    def dcg(res):
        return sum((1 / math.log2(idx + 2)) for idx, (doc_id, _) in enumerate(res) if doc_id in relevantes)

    ideal_relevantes = sorted([(doc_id, 1) for doc_id in relevantes], key=lambda x: x[1], reverse=True)
    ideal_dcg = dcg(ideal_relevantes)
    actual_dcg = dcg(resultados)
    
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

In [38]:
# Aquí usaremos k = 5 para Prec@k, pero puedes ajustar este valor

k = 5  # Precisión en el top-k
metricas_sistemas = {'Sistema_Titulos': {}, 'Sistema_Keywords': {}}

for sistema, resultados in resultados_sistemas.items():
    metricas_sistemas[sistema] = {}
    for consulta_id, resultados_consulta in resultados.items():
        relevantes = juicios_relevancia.get(consulta_id, [])
        
        # Calcular Prec@k
        prec_at_k = precision_at_k(resultados_consulta, relevantes, k)
        
        # Calcular Recall
        recall_value = recall(resultados_consulta, relevantes)
        
        # Calcular F1-score
        f1 = f1_score(prec_at_k, recall_value)
        
        # Calcular Mean Average Precision (MAP)
        map_score = mean_average_precision(resultados_consulta, relevantes)
        
        # Calcular nDCG
        ndcg_score = ndcg(resultados_consulta, relevantes)
        
        # Almacenar métricas para esta consulta
        metricas_sistemas[sistema][consulta_id] = {
            'Prec@k': prec_at_k,
            'Recall': recall_value,
            'F1-score': f1,
            'MAP': map_score,
            'nDCG': ndcg_score
        }

In [39]:
# Imprimir los resultados de las métricas
for sistema, metricas_consulta in metricas_sistemas.items():
    print(f"\nMétricas para {sistema}:")
    for consulta_id, metricas in metricas_consulta.items():
        print(f"\nConsulta {consulta_id}:")
        for metrica, valor in metricas.items():
            print(f"  {metrica}: {valor:.4f}")


Métricas para Sistema_Titulos:

Consulta 1:
  Prec@k: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  MAP: 0.5909
  nDCG: 0.7842

Consulta 2:
  Prec@k: 0.6000
  Recall: 1.0000
  F1-score: 0.7500
  MAP: 1.0000
  nDCG: 1.0000

Consulta 3:
  Prec@k: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  MAP: 0.4338
  nDCG: 0.6966

Consulta 4:
  Prec@k: 0.6000
  Recall: 1.0000
  F1-score: 0.7500
  MAP: 1.0000
  nDCG: 1.0000

Consulta 5:
  Prec@k: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  MAP: 0.4911
  nDCG: 0.7405

Métricas para Sistema_Keywords:

Consulta 1:
  Prec@k: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  MAP: 0.5833
  nDCG: 0.7788

Consulta 2:
  Prec@k: 0.6000
  Recall: 1.0000
  F1-score: 0.7500
  MAP: 1.0000
  nDCG: 1.0000

Consulta 3:
  Prec@k: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  MAP: 0.5076
  nDCG: 0.7482

Consulta 4:
  Prec@k: 0.6000
  Recall: 1.0000
  F1-score: 0.7500
  MAP: 0.9167
  nDCG: 0.9675

Consulta 5:
  Prec@k: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  MAP: 0.5354


#### 4. Análisis y Comparación

In [42]:
# Calcular promedios de métricas para cada sistema
promedios_sistemas = {}

for sistema, metricas_consulta in metricas_sistemas.items():
    total_prec_at_k = total_recall = total_f1 = total_map = total_ndcg = 0
    num_consultas = len(metricas_consulta)
    
    for metricas in metricas_consulta.values():
        total_prec_at_k += metricas['Prec@k']
        total_recall += metricas['Recall']
        total_f1 += metricas['F1-score']
        total_map += metricas['MAP']
        total_ndcg += metricas['nDCG']
    
    # Calcular los promedios
    promedios_sistemas[sistema] = {
        'Prec@k': total_prec_at_k / num_consultas,
        'Recall': total_recall / num_consultas,
        'F1-score': total_f1 / num_consultas,
        'MAP': total_map / num_consultas,
        'nDCG': total_ndcg / num_consultas
    }

# Imprimir los promedios de cada sistema para comparar
print("\nPromedios de métricas por sistema:")
for sistema, promedios in promedios_sistemas.items():
    print(f"\nSistema: {sistema}")
    for metrica, valor in promedios.items():
        print(f"  {metrica}: {valor:.4f}")



Promedios de métricas por sistema:

Sistema: Sistema_Titulos
  Prec@k: 0.3600
  Recall: 1.0000
  F1-score: 0.5000
  MAP: 0.7032
  nDCG: 0.8443

Sistema: Sistema_Keywords
  Prec@k: 0.3600
  Recall: 1.0000
  F1-score: 0.5000
  MAP: 0.7086
  nDCG: 0.8524


#### Discutir cuál sistema es más efectivo y por qué.
1. Prec@k Ambos sistemas tienen un Prec@k de 0.360, lo cual indica que en el conjunto de los primeros reusltados, que los documentos relevantes es la misma.
2. Recall: Lo mismo ambos sistemas lograron un recall de 10000 es decir son iguales.
3. F1-sccore: Ambos sistemas tiene un F1-score de 0.500.
4. MAP: 